In [0]:
import os
import torch
import torch.autograd as autograd
import torch.nn.functional as F
import torch.nn.utils as utils
import torch.optim.lr_scheduler as lr_scheduler
import shutil
import random
import numpy as np

In [0]:
lr = 0.01
init_weight_decay = 0 
epochs = 200 
batch_size = 16
def train_epoch(train_iter, dev_iter, test_iter, model):
  optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay = init_weight_decay)
  steps = 0
  model_count = 0
  model.train()
  for epoch in range(1, epochs+1):
    steps = 0
    print("\n ## The {} Epoch, All {} Epoch ! ##".format(epoch, epochs))
    for batch in train_iter:
      feature, target = batch.text, batch.label
      target = autograd.Variable(target)
      optimizer.zero_grad()
      logit = model(feature)
      #print(target.size(), logit.size())
      loss = F.cross_entropy(logit, target)
      loss.backward()
      optimizer.step()
      
      steps += 1
      if steps % 4 == 0:
        print("hahaha", torch.max(logit, 1)[1])
        corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
        accuracy = float(corrects) / batch_size * 100.0
        print(accuracy)
        
      if steps % 4 == 0:
        print("\n Dev accuracy: ", end="")
        eval(dev_iter, model, epoch)
        #print("Test accuract: ", end="")
    print(steps)
        
def eval(data_iter, model, epoch, test=False):
  model.eval()
  corrects, avg_loss = 0, 0
  for batch in data_iter:
    feature, target = batch.text, batch.label
    logit = model(feature)
    loss = F.cross_entropy(logit, target)
    avg_loss += loss.item()
    
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
  
  size = len(data_iter.dataset)
  avg_loss = avg_loss / size
  accuracy = float(corrects) / size * 100.0
  
  print('Evaluation - loss: {:.6f} acc: {:.4f}%({}/{})'.format(avg_loss, accuracy, corrects, size))
  
  

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import random


"""
    Neural Network: CNN_LSTM
    Detail: the input crosss cnn model and LSTM model independly, then the result of both concat
"""


class CNN_LSTM(nn.Module):
    
    def __init__(self, lstm_hidden_dim, lstm_num_layers, embed_num, embed_dim, class_num, kernel_num, kernel_sizes):
        super(CNN_LSTM, self).__init__()
        self.hidden_dim = lstm_hidden_dim
        self.num_layers = lstm_num_layers
        V = embed_num
        D = embed_dim
        C = class_num
        Ci = 1
        Co = kernel_num
        Ks = kernel_sizes
        self.C = C
        self.embed = nn.Embedding(V, D)
        # pretrained  embedding
        self.embed.weight.data.copy_(vocab.vectors)
        
        # CNN
        self.convs1 = [nn.Conv2d(Ci, Co, (K, D)) for K in Ks]
        self.dropout = nn.Dropout(dropout)

        # LSTM
        self.lstm = nn.LSTM(D, self.hidden_dim, dropout=dropout, num_layers=self.num_layers)

        # linear
        L = len(Ks) * Co + self.hidden_dim
        self.hidden2label1 = nn.Linear(L, L // 2)
        self.hidden2label2 = nn.Linear(L // 2, C)

    def forward(self, x):
        embed = self.embed(x)

        # CNN
        cnn_x = embed
        cnn_x = torch.transpose(cnn_x, 0, 1)
        cnn_x = cnn_x.unsqueeze(1)
        cnn_x = [F.relu(conv(cnn_x)).squeeze(3) for conv in self.convs1]  # [(N,Co,W), ...]*len(Ks)
        cnn_x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in cnn_x]  # [(N,Co), ...]*len(Ks)
        cnn_x = torch.cat(cnn_x, 1)
        cnn_x = self.dropout(cnn_x)

        # LSTM
        lstm_x = embed.view(len(x), embed.size(1), -1)
        lstm_out, _ = self.lstm(lstm_x)
        lstm_out = torch.transpose(lstm_out, 0, 1)
        lstm_out = torch.transpose(lstm_out, 1, 2)
        lstm_out = F.max_pool1d(lstm_out, lstm_out.size(2)).squeeze(2)

        # CNN and LSTM cat
        cnn_x = torch.transpose(cnn_x, 0, 1)
        lstm_out = torch.transpose(lstm_out, 0, 1)
        cnn_lstm_out = torch.cat((cnn_x, lstm_out), 0)
        cnn_lstm_out = torch.transpose(cnn_lstm_out, 0, 1)

        # linear
        cnn_lstm_out = self.hidden2label1(F.tanh(cnn_lstm_out))
        cnn_lstm_out = self.hidden2label2(F.tanh(cnn_lstm_out))

        # output
        logit = cnn_lstm_out
        return logit

In [0]:
import numpy as np
import torchtext
from torchtext import data
from torchtext.vocab import Vectors
from tqdm import tqdm

In [6]:
import pandas as pd
train=pd.read_csv('train.tsv', sep='\t')
print(train.head())

train = train.drop(columns=['PhraseId', 'SentenceId'])
print(train)

with open('train_drop.tsv','w') as f:
  f.write(train.to_csv(sep='\t', index=False))


   PhraseId  ...  Sentiment
0         1  ...          1
1         2  ...          2
2         3  ...          2
3         4  ...          2
4         5  ...          2

[5 rows x 4 columns]
                                                   Phrase  Sentiment
0       A series of escapades demonstrating the adage ...          1
1       A series of escapades demonstrating the adage ...          2
2                                                A series          2
3                                                       A          2
4                                                  series          2
...                                                   ...        ...
156055                                          Hearst 's          2
156056                          forced avuncular chortles          1
156057                                 avuncular chortles          3
156058                                          avuncular          2
156059                                           ch

In [7]:


tokenize = lambda x: x.split()
text = data.Field(sequential=True, tokenize=tokenize, lower=True, fix_length=200)
#postprocessing=data.Pipeline(int)
label = data.Field(sequential=False, use_vocab=False)

train, test = data.TabularDataset.splits(path='.', train='train_drop.tsv', test='test.tsv', format='tsv', fields=[('text', text),('label', label)], filter_pred=lambda ex: ex.label in ['0', '1','2','3','4'])
text.build_vocab(train, vectors="glove.6B.100d")

train, val = data.Dataset.split(train, split_ratio=0.8)

train_iter, val_iter, test_iter = data.Iterator.splits((train, val, test), sort_key=lambda x: len(x.text), batch_size=16)#自动pad

import torch.nn as nn
vocab = text.vocab
#embed = nn.Embedding(len(vocab), 100)
#embed.weight.data.copy_(vocab.vectors)



.vector_cache/glove.6B.zip: 862MB [06:26, 2.23MB/s]                           
100%|█████████▉| 398405/400000 [00:16<00:00, 25242.35it/s]

In [8]:
embed_dim = 100
lstm_hidden_dim = 300
lstm_num_layers = 1
dropout = 0.75
dropout_embed = 0.75

kernel_num = 300
kernel_sizes = 3,4,5

embed_num = len(text.vocab)
class_num = 5


model = CNN_LSTM(lstm_hidden_dim, lstm_num_layers, embed_num, embed_dim, class_num, kernel_num, kernel_sizes)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:51: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.75 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [0]:
train_epoch(train_iter, val_iter, test_iter, model)


 ## The 1 Epoch, All 200 Epoch ! ##


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


hahaha tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
18.75

 Dev accuracy: Evaluation - loss: 0.170943 acc: 0.0064%(2/31212)
hahaha tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
50.0

 Dev accuracy: Evaluation - loss: 0.110056 acc: 0.0032%(1/31212)
hahaha tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])
31.25

 Dev accuracy: Evaluation - loss: 0.107857 acc: 0.0160%(5/31212)
hahaha tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])
43.75

 Dev accuracy: 

In [0]:
vars(train[0])

{'label': '2', 'text': ['guessed']}

In [0]:
vars(next(iter(train_iter)))['text']

tensor([[15799,    11,    19,  ...,    65,   484,  4375],
        [    1,    44,   366,  ...,  3370,     3,     1],
        [    1,  9442, 14140,  ...,    32,   484,     1],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]])